<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/complete_end_to_end_sympy_ode_simulator_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# complete_end_to_end_sympy_ode_simulator.py
# A single-file, ready-to-run script that:
# 1) Builds a simple symbolic "theory" from a directed graph (nodes, edges)
# 2) Interprets each equation's LHS as an ODE derivative for each state
# 3) Substitutes parameter symbols with numeric values
# 4) Integrates with scipy.integrate.solve_ivp and prints results

from __future__ import annotations

from typing import Dict, List, Tuple, Optional, Iterable
import numpy as np
import sympy as sp
from scipy.integrate import solve_ivp


def build_symbolic_theory(
    nodes: Iterable[str],
    edges: Iterable[Tuple[str, str]],
    var_prefix: str = "v",
    add_constant: bool = False
) -> Dict:
    """
    Build a linear symbolic "theory" from a directed graph.

    For each node d in nodes, construct an equation:
        0 = sum_{(s->d) in edges} a_s_d * v_s  -  b_d * v_d  +  [c_d if add_constant]
    where:
        - v_n are state variables (one per node n)
        - a_s_d, b_d, c_d are symbolic parameters (coefficients, damping, constants)

    Returns:
        {
            'equations': { sym(v_d): Eq( expr_d(v), 0 ), ... },
            'var_map'  : { node_name: sym(v_node_name), ... },
            'params'   : set of parameter symbols used
        }
    """
    nodes = list(nodes)
    edges = list(edges)

    # Create a sympy symbol for each node's state variable
    var_map: Dict[str, sp.Symbol] = {n: sp.Symbol(f"{var_prefix}_{n}") for n in nodes}

    # Helper to create parameter symbols with consistent names
    def a_sym(s: str, d: str) -> sp.Symbol:
        return sp.Symbol(f"a_{s}_{d}")

    def b_sym(d: str) -> sp.Symbol:
        return sp.Symbol(f"b_{d}")

    def c_sym(d: str) -> sp.Symbol:
        return sp.Symbol(f"c_{d}")

    # Build per-node expressions
    eq_map: Dict[sp.Symbol, sp.Equality] = {}
    params = set()

    # Precompute incoming adjacency
    incoming: Dict[str, List[str]] = {n: [] for n in nodes}
    for s, d in edges:
        if d not in incoming:
            raise ValueError(f"Edge points to unknown node '{d}'. Known nodes: {nodes}")
        if s not in var_map:
            raise ValueError(f"Edge comes from unknown node '{s}'. Known nodes: {nodes}")
        incoming[d].append(s)

    for d in nodes:
        vd = var_map[d]
        expr = sp.Integer(0)

        # Sum of incoming influences
        for s in incoming[d]:
            asd = a_sym(s, d)
            vs = var_map[s]
            params.add(asd)
            expr += asd * vs

        # Self-damping term (negative sign to promote stability)
        bd = b_sym(d)
        params.add(bd)
        expr += -bd * vd

        # Optional constant driving term
        if add_constant:
            cd = c_sym(d)
            params.add(cd)
            expr += cd

        # Store as an equality expr == 0 (we will use LHS as derivative)
        eq_map[vd] = sp.Eq(sp.simplify(expr), 0)

    return {
        "equations": eq_map,   # mapping: state_symbol -> Eq(lhs, 0)
        "var_map": var_map,    # mapping: node_name -> state_symbol
        "params": params,      # set of parameter symbols
    }


class SympyODESimulator:
    """
    Turn a symbolic 'theory' (linear constraints) into an ODE system by interpreting
    each equation's LHS as a derivative for the corresponding state variable.

    Assumptions:
    - theory['equations']: dict { state_symbol: sympy Eq(expr, 0) }
    - theory['var_map']: dict { node_name: state_symbol }
    - Coefficients/constants named:
        a_*_*  -> edge weights      (substitute to coeff_value)
        b_*    -> self-damping      (substitute to coeff_value; equation has -b_* v term)
        c_*    -> constant driving  (substitute to const_value)
    """

    def __init__(
        self,
        theory: Dict,
        coeff_value: float = 1.0,
        const_value: float = 0.0,
        state_order: Optional[List[str]] = None
    ):
        self.theory = theory

        # Validate structure
        if not isinstance(theory.get("equations"), dict):
            raise TypeError("theory['equations'] must be a dict {state_symbol: Eq(lhs, 0)}")

        self.eq_map: Dict[sp.Symbol, sp.Equality] = theory["equations"]
        self.node_to_sym: Dict[str, sp.Symbol] = theory["var_map"]
        self.sym_to_node: Dict[sp.Symbol, str] = {v: k for k, v in self.node_to_sym.items()}

        # Determine state order (names are node names; variables are v_<name>)
        if state_order is None:
            self.state_names = sorted(self.node_to_sym.keys())
        else:
            missing = [n for n in state_order if n not in self.node_to_sym]
            if missing:
                raise ValueError(f"Unknown names in state_order: {missing}")
            self.state_names = list(state_order)

        self.state_syms: Tuple[sp.Symbol, ...] = tuple(self.node_to_sym[n] for n in self.state_names)

        # Substitute numeric values for parameter symbols
        subs: Dict[sp.Symbol, float] = {}
        for sym in set().union(*[eq.free_symbols for eq in self.eq_map.values()]):
            name = str(sym)
            if name.startswith("a_") or name.startswith("b_"):
                subs[sym] = float(coeff_value)
            elif name.startswith("c_"):
                subs[sym] = float(const_value)

        # Construct derivative expressions in the chosen state order
        self.f_exprs: List[sp.Expr] = []
        for s in self.state_syms:
            if s not in self.eq_map:
                raise ValueError(f"No equation provided for state symbol {s} ({self.sym_to_node.get(s, '?')})")
            expr = sp.simplify(self.eq_map[s].lhs.subs(subs))
            self.f_exprs.append(expr)

        # Lambdify each derivative as a function of the state vector (positional)
        self.f_funcs = [sp.lambdify(self.state_syms, f, modules="numpy") for f in self.f_exprs]

    def _ode(self, t: float, y: np.ndarray) -> np.ndarray:
        vals = tuple(float(v) for v in y)
        dydt = np.array([float(f(*vals)) for f in self.f_funcs], dtype=float)
        return dydt

    def integrate(
        self,
        y0: np.ndarray,
        t_span: Tuple[float, float] = (0.0, 10.0),
        t_eval: Optional[np.ndarray] = None,
        rtol: float = 1e-6,
        atol: float = 1e-8,
        method: str = "RK45"
    ):
        y0 = np.asarray(y0, dtype=float)
        if y0.shape != (len(self.state_syms),):
            raise ValueError(f"y0 must have shape ({len(self.state_syms)},), got {y0.shape}")
        return solve_ivp(self._ode, t_span, y0, t_eval=t_eval, rtol=rtol, atol=atol, method=method)

    @property
    def states(self) -> List[str]:
        return list(self.state_names)


if __name__ == "__main__":
    # Define a small directed graph: x -> y (x drives y), both with self-damping
    nodes = ["x", "y"]
    edges = [("x", "y")]  # add ("y", "x") to make it bidirectional

    # Build symbolic theory
    theory = build_symbolic_theory(
        nodes=nodes,
        edges=edges,
        var_prefix="v",
        add_constant=False,  # set True to add constant driving terms c_<node>
    )

    # Create simulator
    sim = SympyODESimulator(
        theory=theory,
        coeff_value=1.0,   # a_*, b_* -> 1.0 (note: equation uses -b_* v for damping)
        const_value=0.0,   # c_*      -> 0.0
        state_order=["x", "y"],  # order of states in the ODE and y0
    )

    # Initial conditions for [v_x, v_y]
    y0 = np.array([1.0, -1.0], dtype=float)

    # Time span and evaluation grid
    t_span = (0.0, 5.0)
    t_eval = np.linspace(t_span[0], t_span[1], 101)

    # Integrate
    sol = sim.integrate(y0, t_span=t_span, t_eval=t_eval)

    # Report results
    print("States order:", sim.states)
    print("t (first 5):", sol.t[:5])
    print("y shape (states x time):", sol.y.shape)
    print("y[:, :5] (first 5 timepoints):")
    print(np.round(sol.y[:, :5], 4))

    # Quick sanity insights
    # With edges=[("x","y")] and coeff_value=1:
    # dv_x/dt = -1 * v_x
    # dv_y/dt = 1 * v_x - 1 * v_y
    # So v_x decays to 0, and v_y is driven by v_x while also decaying.